# PARTE 1 : Oficinas de chicago

# 1) Ponderación de los criterios a seleccionar


Remisión al readme

# 2)  Importaciones varias 

In [3]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import requests
import json
import os
import pandas as pd
import regex as re

In [4]:
from iteration_utilities import duplicates
from iteration_utilities import unique_everseen

# 3) Importación del dataset

Conexión con servidor

In [5]:
client = MongoClient("localhost:27017")
db = client.get_database("Ironhack")

Conexión base de datos

In [6]:
db = client.get_database("Ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [7]:
collection = db.get_collection('localhost:27017')

Pequeños análisis

In [8]:
#collection

In [9]:
collection.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [10]:
#db.companies.find_one({})

# 4) Creación dataframe


In [11]:
cond1 = {"offices.city" : "Chicago"}
cond2 = {"offices.longitude":{"$exists": True}}
cond3 = {"offices.latitude":{"$exists": True}}
proj = {"_id":0, "offices":1}



In [12]:
guay = list(collection.find({"$and": [cond1,cond2,cond3]}, proj))

In [13]:
chicago_offices = []
for doc in guay:
    for of in doc["offices"]:
        if of["city"] == "Chicago":
            chicago_offices.append(of)

In [14]:
datachicago = pd.DataFrame(chicago_offices)

In [15]:
datachicago

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Chicago Office,225 West Washington,Ste. 2200,60606,Chicago,IL,USA,NaN,NaN
1,None,180 North Stetson Avenue,Suite 3500,60601,Chicago,IL,USA,41.885619,-87.622309
2,Google Chicago,20 West Kinzie St.,,60610,Chicago,IL,USA,41.889474,-87.628912
3,None,20 W Kinzie,9th Floor,60610,Chicago,IL,USA,41.889474,-87.628912
4,Chicago,"311 W. Superior Street, Suite 209",,60654,Chicago,IL,USA,NaN,NaN
...,...,...,...,...,...,...,...,...,...
201,,#127 Suite 134,Thrikkakara,,Chicago,IL,USA,41.663359,-87.625905
202,,,,,Chicago,IL,USA,41.879535,-87.624333
203,,222 Merchandise Mart Plaza,Suite 900,60654,Chicago,IL,USA,41.887919,-87.635425
204,,311 West Monroe Street,Suite 1000,60606,Chicago,IL,USA,41.880626,-87.635607


#### Obtención de la columna nombre compañías

In [33]:
guay = list(collection.find({}))

In [36]:
type(guay)

list

In [17]:
chicago_names = []
for doc in guay:
    for of in doc["offices"]:
        if of["city"] == "Chicago":
            chicago_names.append(doc["name"])

In [18]:
datachicago2 = pd.DataFrame(chicago_names)

In [35]:
datachicago2

,0
0,AddThis
1,Info
2,Google
3,FeedBurner
4,Meebo
...,...
201,Hiremac
202,Midway Games
203,CCC Information Services
204,optionsXpress


In [20]:
data = pd.concat([datachicago, datachicago2], axis=1)

In [21]:
data

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,0
0,Chicago Office,225 West Washington,Ste. 2200,60606,Chicago,IL,USA,NaN,NaN,AddThis
1,None,180 North Stetson Avenue,Suite 3500,60601,Chicago,IL,USA,41.885619,-87.622309,Info
2,Google Chicago,20 West Kinzie St.,,60610,Chicago,IL,USA,41.889474,-87.628912,Google
3,None,20 W Kinzie,9th Floor,60610,Chicago,IL,USA,41.889474,-87.628912,FeedBurner
4,Chicago,"311 W. Superior Street, Suite 209",,60654,Chicago,IL,USA,NaN,NaN,Meebo
...,...,...,...,...,...,...,...,...,...,...
201,,#127 Suite 134,Thrikkakara,,Chicago,IL,USA,41.663359,-87.625905,Hiremac
202,,,,,Chicago,IL,USA,41.879535,-87.624333,Midway Games
203,,222 Merchandise Mart Plaza,Suite 900,60654,Chicago,IL,USA,41.887919,-87.635425,CCC Information Services
204,,311 West Monroe Street,Suite 1000,60606,Chicago,IL,USA,41.880626,-87.635607,optionsXpress


# 5 ) Limpieza Dataframe.

Será útil para extraer la longitud y la latitud.

Limpieza de columnnas no ncesarias

In [22]:
data.drop(['address1', "description", 'address2', 'zip_code', 'state_code', "country_code"], axis=1, inplace=True, )

In [23]:
data

,city,latitude,longitude,0
0,Chicago,NaN,NaN,AddThis
1,Chicago,41.885619,-87.622309,Info
2,Chicago,41.889474,-87.628912,Google
3,Chicago,41.889474,-87.628912,FeedBurner
4,Chicago,NaN,NaN,Meebo
...,...,...,...,...
201,Chicago,41.663359,-87.625905,Hiremac
202,Chicago,41.879535,-87.624333,Midway Games
203,Chicago,41.887919,-87.635425,CCC Information Services
204,Chicago,41.880626,-87.635607,optionsXpress


Eliminamos todas las filas cuya longitud o latitud sean cero 

In [24]:
data.isnull().sum(axis = 0)

city          0
latitude     69
longitude    69
0             0
dtype: int64

In [25]:
data

,city,latitude,longitude,0
0,Chicago,NaN,NaN,AddThis
1,Chicago,41.885619,-87.622309,Info
2,Chicago,41.889474,-87.628912,Google
3,Chicago,41.889474,-87.628912,FeedBurner
4,Chicago,NaN,NaN,Meebo
...,...,...,...,...
201,Chicago,41.663359,-87.625905,Hiremac
202,Chicago,41.879535,-87.624333,Midway Games
203,Chicago,41.887919,-87.635425,CCC Information Services
204,Chicago,41.880626,-87.635607,optionsXpress


In [26]:
data = data.dropna()

In [27]:
data

,city,latitude,longitude,0
1,Chicago,41.885619,-87.622309,Info
2,Chicago,41.889474,-87.628912,Google
3,Chicago,41.889474,-87.628912,FeedBurner
5,Chicago,41.889218,-87.655723,37signals
10,Chicago,41.881769,-87.638752,Parlano
...,...,...,...,...
200,Chicago,41.880169,-87.624926,Sky Bird Travel Tours
201,Chicago,41.663359,-87.625905,Hiremac
202,Chicago,41.879535,-87.624333,Midway Games
203,Chicago,41.887919,-87.635425,CCC Information Services


In [28]:
data.isnull().sum(axis = 0)

city         0
latitude     0
longitude    0
0            0
dtype: int64

In [38]:
data=data.rename(columns = { 0 :'name_companie'})

In [39]:
data

,city,latitude,longitude,name_companie
1,Chicago,41.885619,-87.622309,Info
2,Chicago,41.889474,-87.628912,Google
3,Chicago,41.889474,-87.628912,FeedBurner
5,Chicago,41.889218,-87.655723,37signals
10,Chicago,41.881769,-87.638752,Parlano
...,...,...,...,...
200,Chicago,41.880169,-87.624926,Sky Bird Travel Tours
201,Chicago,41.663359,-87.625905,Hiremac
202,Chicago,41.879535,-87.624333,Midway Games
203,Chicago,41.887919,-87.635425,CCC Information Services


# 6) Exportación del dataframe

In [31]:
chicago_data = data
chicago_data.to_csv("./chicago_data.csv") #Exportación del DataFrame